In [1]:
# you don't need to run this if you use 'pip install -r requirements'

# !pip install PyPDF2
# !python -m spacy download fr_core_news_sm
# !pip install pikepdf
# !pip install PyMuPDF
# !pip install langchain
# !pip install Unstructured

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00


In [5]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity
import PyPDF2
import pikepdf
import fitz

In [15]:
nlp = spacy.load('fr_core_news_sm')

# Define the maximum number of tokens for the output sentences
max_tokens = 10

In [16]:
# Function to extract text from a PDF file
def extract_text_from_pdf(file_path):
    doc = fitz.open(file_path)
    text = ''
    for i in range(doc.page_count):
      text += doc.get_page_text(i)
    return text 

In [17]:
text = extract_text_from_pdf('../ressources/loi_english.pdf')

In [18]:
pdf_file = open('../ressources/loi_english.pdf', 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)

text_content = ''
for page in range(len(pdf_reader.pages)):
    text_content += pdf_reader.pages[page].extract_text()

# Split the text content into individual sentences
sentences = text_content.split('.')

In [19]:
input_sentence = "what if i witness a crime and i was blackmailed"

In [23]:
import PyPDF2


with open('../ressources/loi_english.pdf', 'rb') as pdf_file:
   
    pdf_reader = PyPDF2.PdfReader(pdf_file)
   
   
    articles = []
   
   
    num_pages = len(pdf_reader.pages)
   
   
    for page in range(5,num_pages):
       
        page_obj = pdf_reader.pages[page]
        page_text = page_obj.extract_text()
       
       
        article_list = page_text.split('Section ')
        article_list.pop(0)  
       
       
        for article in article_list:
            articles.append('Section ' + article)
           
   
    print(articles)

['Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Section ', 'Se

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader

In [31]:
loader = UnstructuredPDFLoader('../ressources/loi_english.pdf')

In [32]:
data = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [53]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 700, chunk_overlap = 0)

texts = text_splitter.split_documents(data)

In [54]:
texts[0].page_content[40:]

'\nThe courts and tribunals will continue to observe the laws and regulations\n\nparticulars governing all matters not regulated by the code.\n\nHowever, these courts will only be able to pronounce penalties falling within the categories provided for by him and according to the distinctions enacted in his article 5 below.\n\nSection 4\n\nThe provisions of this code apply even to matters governed by specific laws and regulations in all that has not been expressly provided for in these laws.\n\nSection 5'

In [55]:
articles2 = [texts[i].page_content[40:] for i in range(len(texts))]

In [56]:
import pickle

with open("../ressources/paragraphs_LEGAL_ARTICLES2.pkl", "wb") as f:
    pickle.dump(articles2, f)

In [57]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix, save_npz

def find_similar_paragraphs(paragraphs, input_sentence, max_tokens=100):
    # Tokenize the input sentence
    input_tokens = input_sentence.split()[:max_tokens]
    
    # Compute the TF-IDF matrix for the paragraphs
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(paragraphs)
    # print(type(tfidf_matrix))
    save_npz("../ressources/matrix_NLP_SPACY2.npz", tfidf_matrix)
    
    # Compute the cosine similarity between the input sentence and each paragraph
    input_tfidf = vectorizer.transform([' '.join(input_tokens)])
    similarities = cosine_similarity(input_tfidf, tfidf_matrix)
    
    # Find the indices of the two most similar paragraphs
    indices = np.argsort(similarities, axis=1)[:, -2:]
    
    # Return the two most similar paragraphs
    return [paragraphs[i] for i in indices[0]]


In [58]:
paragraphs = articles2
input_sentence = "abandon their born or unborn child"
similar_paragraphs = find_similar_paragraphs(paragraphs, input_sentence, max_tokens=5)
print(similar_paragraphs)


[' to one year and a fine of 250 to 5,000 dirhams, whoever, with the intention of undermining their honor, their delicacy or the respect due to their authority, insults in the exercise of their functions or on the occasion of this exercise, a magistrate, a public official, a commander or agent of the public force, either by words, gestures, threats, sending or delivery of any object, either in writing or drawing not made public.\n\nWhen the contempt of one or more magistrates or juror assessors is committed at the hearing of a court or tribunal, the imprisonment is from one to two years.', 'abandon their born or unborn child;\n\n2° Brings or tries to bring his intermediary to make collect or\n\nadopt a born or unborn child.\n\nSection 467\n\nIs punishable by imprisonment of one to six months and a fine of 200,186\n\nto 5,000 dirhams any person who: 1° Causes\n\nor attempts to have signed, by the future parents or one of them, an act\n\nunder which they undertake to give up an unborn ch